In [1]:
# Importing required modules
import warnings
import gc
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch
import cv2
import pandas as pd
import numpy as np
import pathlib
import os,sys
from torchvision.models import resnet50
import wandb
from sklearn import preprocessing
warnings.filterwarnings('ignore')

/opt/conda/envs/akash/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Seeding to help make results reproduceable
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
# Building custom dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, X_train, y_train, transform):
        self.root_dir = root_dir
        self.transform = transform
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, index):
        label = self.y_train.iloc[index]
        image_path = self.X_train.iloc[index]
        image = np.load(f'{root_dir}/{image_path}')
        image = np.squeeze(image)
        image = np.uint8(image * 255)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        image = self.transform(image)
        return image, torch.tensor(label)

In [4]:
def run():
    torch.cuda.empty_cache()
    seed_everything(SEED)

    #  Loading the train data
    train_df = pd.read_csv(train_csv_path)
    # train_df = train_df.sample(500)
    train_df.head()
    le = preprocessing.LabelEncoder()
    train_df['label'] = le.fit_transform(train_df['label'])

    # building training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(train_df['path'], train_df['label'], test_size=0.1, random_state=SEED)
    print('Data lengths: ', len(X_train), len(X_valid), len(y_train), len(y_valid))

    # Data transforms
    train_transforms = transforms.Compose([transforms.ToPILImage(),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])
    test_transforms = transforms.Compose([transforms.ToPILImage(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])

    # DataLoader
    train_dataset = CustomDataset(root_dir,X_train, y_train,train_transforms)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
    valid_dataset = CustomDataset(root_dir,X_valid, y_valid,test_transforms)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)

    # Loading the pretrained model here
    if 'efficientnet' in model_name:
        model = EfficientNet.from_pretrained(model_name)
        # print(model)
        model._fc = nn.Sequential(
            nn.Linear(mid_features, 100),
            nn.ReLU(),
            nn.Linear(100, 3)
        )
    else:
        model = resnet50(pretrained = True)
        model._fc = nn.Sequential(
            nn.Linear(mid_features, 100),
            nn.ReLU(),
            nn.Linear(100, 3)
        )
       
    for params in model.parameters():
        params.requires_grad = True

    # Training model
    epoch = 0
    best_s = 0.0
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma_value)
    EPOCHS = num_epoch
    
    while epoch < num_epoch:
        print(f'Epoch: {epoch+1}/{EPOCHS}')

        correct = 0
        total = 0
        losses = []
        # print("check1")
        for batch_idx, data in enumerate(tqdm(train_loader, total=len(train_loader))):
            images, targets = data
            images = images.to(device)
            targets = targets.to(device)
            output = model(images)
            loss = criterion(output, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, pred = torch.max(output, 1)
            correct += (pred == targets).sum().item()
            total += pred.size(0)
            losses.append(loss.item())
            loss.detach()
            del images, targets, output, loss
            gc.collect()

        train_loss = np.mean(losses)
        train_acc = correct * 100.0 / total
        del losses
        total = 0
        correct = 0
        valid_acc = 0

        with torch.no_grad():
            for batch_idx, data in enumerate(tqdm(valid_loader, total=len(valid_loader))):
                images, targets = data
                images = images.to(device)
                targets = targets.to(device)
                output = model(images)
                _, pred = torch.max(output, 1)
                correct += (pred == targets).sum().item()
                total += pred.size(0)
        valid_acc = correct/total * 100

        #Saving checkpoint, each time the current valid_acc overshoots the previous best.
        if valid_acc > best_s:
            checkpoint_name = 'checkpoint_' + str(epoch+1) + f'_{model_name}' + '.pth.tar'
            torch.save({'model_state_dict': model.state_dict(),}, checkpoint_name)
            best_s = valid_acc
        print(f'Train Loss: {train_loss}\tTrain Acc: {train_acc}\tLR: {scheduler.get_lr()}\tValid Accuracy: {valid_acc}', end = '\r')
        if(bool_wandb):
            wandb.log({"Train loss": train_loss, "Train Acc": train_acc, "Learning Rate": (scheduler.get_lr()[0]), "Valid Accuracy": valid_acc})
        scheduler.step()
        epoch+=1

In [5]:
# This helps make all other paths relative
base_path = pathlib.Path().absolute()
bool_wandb = False
# Model choice
# arg:  efficientnet-b0, efficientnet-b3, efficientnet-b4, efficientnet-b5, efficientnet-b6, efficientnet-b7, resnet50
model_name = 'efficientnet-b6'

# Input of the required hyperparameters
BATCH_SIZE = 128
# 1280, 1536, 1792, 2048, 2304, 2048
mid_features = 2304
learning_rate = 0.005
gamma_value = 0.9

SEED = 42
num_epoch = 100
num_workers = 2
device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
root_dir = base_path
if not os.path.exists(f'{root_dir}/dataset'):
    print("Dataset missing.")
    print(f'{root_dir}/dataset')
    sys.exit()
train_csv_path = f'{base_path}/data.csv'

if bool_wandb:
    # user_secrets = UserSecretsClient()
    # wandb_api_key = user_secrets.get_secret("wandb_api_key")
    # wandb.login(key=wandb_api_key)
    wandb.login()
    wandb.init(project="deeplenses_task-1", entity="gakash2001")
    wandb.config = {
        'model' : model_name,
        "learning_rate": learning_rate,
        "epochs": num_epoch,
        "batch_size": BATCH_SIZE
    }
run()

Data lengths:  27000 3000 27000 3000
Loaded pretrained weights for efficientnet-b6
Epoch: 1/100


100%|██████████| 24/24 [00:03<00:00,  6.95it/s]


Epoch: 2/1001.0841647256607128	Train Acc: 37.25925925925926	LR: [0.005]	Valid Accuracy: 53.333333333333336


100%|██████████| 24/24 [00:03<00:00,  6.93it/s]


Epoch: 3/1000.5889797486117666	Train Acc: 74.82962962962964	LR: [0.004050000000000001]	Valid Accuracy: 86.66666666666667


...

100%|██████████| 24/24 [00:03<00:00,  6.89it/s]


Epoch: 88/100.0012684717183637727	Train Acc: 99.96296296296296	LR: [5.224783816588932e-07]	Valid Accuracy: 94.93333333333334


100%|██████████| 24/24 [00:03<00:00,  6.93it/s]


Epoch: 89/100.0008149039356544671	Train Acc: 99.97407407407407	LR: [4.702305434930039e-07]	Valid Accuracy: 94.86666666666666


  2%|▏         | 5/211 [00:03<01:55,  1.79it/s]